In [1]:
import torch
import torch.nn as nn

In [ ]:
class ModalityProjector(nn.Module):
    def __init__(self):
        super().__init__()
        self.proj = nn.Linear(, 64, bias=False)
        self.apply(self._init_weights)

        self.scale_factor = 2
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
    

    def pixel_shuffle(self, x):
        bsz, seq, embed_dim = x.size()
        seq_root = int(seq**0.5)

        height = width = seq_root
        print(height, width)
        x = x.view(bsz, height, width, embed_dim)

        h_out = height // self.scale_factor
        w_out = width // self.scale_factor

        x = x.reshape(bsz, h_out, self.scale_factor, w_out, self.scale_factor, embed_dim)
        x = x.permute(0, 1, 3, 2, 4, 5).contiguous()
        x = x.reshape(bsz, h_out * w_out, embed_dim * self.scale_factor**2)

        return x
    
    def forward(self, x):
        x = self.pixel_shuffle(x)
        print(x.shape)
        return self.proj(x)

In [18]:
mp = ModalityProjector()

In [19]:
mp(torch.randn(1, 64, 32)).shape

8 8
torch.Size([1, 16, 128])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x128 and 32x64)